<a href="https://colab.research.google.com/github/WonMin13/EnterpriseDataBase/blob/main/Lab%20Work%20%232/EDB_lab_2_%D0%94%D1%83%D0%B1%D0%BC%D0%B0%D0%BD_6133.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №2 Машинное обучение по курсу "Databases in Enterprise Systems"

Выполнил студент группы 6133-010402D Дубман Любовь

## Задание на лабораторную работу

1. Сформулировать задачу машинного обучения для БД из 1 лабораторной, которую собираетесь решать.

2. Разделить данные из БД на тренировочные и тестовые.
3. Определить вектор признаков для данных вашей предметной области. Самая сложная часть работы, от которой зависит дальнейшая эффективность машинного обучения, советую ориентироваться на существующие принципы выбора признаков и предварительной обработки данных в выбранной предметной области.
4. Выбрать тип классификатора: от самых простых kNN и линейного до нейронных сетей и RandomForest.
5. Оценить оптимальные значения гиперпараметров. Построить соответствующие зависимости качества распознавания от гиперпараметров.
6. Обучить классификатор на тренировочной выборке и оценить его эффективность на тестовой выборке.

## Выполнение лабораторной работы

### Этап 0

Задача машинного обучения состоит в разработке модели, способной предсказывать наличие или отсутствие диабета у пациентов на основе клинических данных. Для этого используются данные, включающие в себя информацию о беременности, уровне глюкозы, кровяном давлении, состоянии кожи, уровне инсулина, индексе массы тела, наличии родословной диабета и возрасте пациента. Модель должна обучаться на исторических данных, после чего её эффективность будет оцениваться на новых наблюдениях, позволяя определять риск развития диабета у пациентов и принимать соответствующие меры профилактики или лечения.

### Этап 1

**Подключение бибилотек**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score
from warnings import filterwarnings
filterwarnings("ignore")

**Подключение Google Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Загрузка данных**

In [3]:
dataframe = pd.read_csv("/content/drive/MyDrive/datasets/diabetes.csv")
dataframe

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


### Этап 2

**Выделение вектора признаков**

Перед тем, как выполнить разделение данных, рассмотрим используемый набор данных. Это набор данных по диабету. В наборе учавствует множество данных, таких как:

* Беременность,
* Уровень глюкозы,
* Кровяное давление,
* Состояние кожи,
* Инсулин,
* ИМТ,
* Родословная диабета,
* Возраст,
* Результат.

[Ссылка на Dataset](https://www.kaggle.com/datasets/pritsheta/diabetes-dataset).



Совершенно очевидным становится тот факт, что в качестве признаков могут и должны быть использованы все столбцы, кроме крайнего. На основании данного принципа и будем производить разбиение даных.

In [4]:
X_data = dataframe.drop("Outcome", axis=1)
y_target = dataframe["Outcome"]

**Разделение данных на тестовую и тренировочную выборки**

Для разделения данных на тренировочные и тестовые выборки воспользуемся стандартной командой и произведем разбиение в слетующем процентном соотношении 30/70 (30 - тестовая выборка, 70 - обучающая).

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=.35, random_state=42)

**Подготовка нормализованного набора данных**

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.fit_transform(X_test)

### Этап 3. Выбор классификатора

**kNN**

In [7]:
classifierKNN = KNeighborsClassifier()
classifierKNN.fit(X_train_scaler, y_train)
classifierPredictionKNN = classifierKNN.predict(X_test_scaler)

In [8]:
print("Accuracy KNeighborsClassifier:", accuracy_score(y_test, classifierPredictionKNN) * 100)

Accuracy KNeighborsClassifier: 71.74721189591078


**DecisionTreeClassifier**

In [9]:
classifierDTC = DecisionTreeClassifier()
classifierDTC.fit(X_train_scaler, y_train)
classifierPredictionDTC = classifierDTC.predict(X_test_scaler)

In [10]:
print("Accuracy DecisionTreeClassifier:", accuracy_score(y_test, classifierPredictionDTC)*100)

Accuracy DecisionTreeClassifier: 72.4907063197026


**RandomForestClassifier**

In [11]:
classifierRF = RandomForestClassifier()
classifierRF.fit(X_train_scaler, y_train)
classifierPredictionRF = classifierRF.predict(X_test_scaler)

In [12]:
print("Accuracy RandomForestClassifier:", accuracy_score(y_test, classifierPredictionRF) * 100)

Accuracy RandomForestClassifier: 75.09293680297398


### Этап 4. Определение оптимальных значений гиперпараметров

**kNN**

In [13]:
KNeighborsParams = {'n_neighbors': range(1, 32)}
KNeighborsGSCV = GridSearchCV(classifierKNN, KNeighborsParams, scoring="roc_auc")

**DecisionTreeClassifier**

In [14]:
DecisionTreeParams = {
    "max_depth": range(1, 33),
    "min_samples_split": np.linspace(0.01, 0.1, 10, endpoint=True),
    "min_samples_leaf": np.linspace(0.01, 0.1, 10, endpoint=True),
}
DecisionTreeGSCV = GridSearchCV(classifierDTC, DecisionTreeParams)

**RandomForestClassifier**

In [15]:
RandomForestParams = {"n_estimators": range(1, 32)}
RandomForestGSCV = GridSearchCV(classifierRF, RandomForestParams)

### Этап 5. Выбор классификатора с учетом гиперпараметров

**kNN**

In [16]:
KNeighborsGSCV.fit(X_train_scaler, y_train)
print(KNeighborsGSCV.best_estimator_)
KNeighborsPredictGSCV = KNeighborsGSCV.predict(X_test_scaler)
print("Accuracy KNeighborsClassifier with optimal hyperparameters:", accuracy_score(y_test, KNeighborsPredictGSCV) * 100)

KNeighborsClassifier(n_neighbors=31)
Accuracy KNeighborsClassifier with optimal hyperparameters: 75.09293680297398


**DecisionTreeClassifier**

In [17]:
DecisionTreeGSCV.fit(X_train_scaler, y_train)
print(DecisionTreeGSCV.best_estimator_)
DecisionTreeGSCV_Predict = DecisionTreeGSCV.predict(X_test_scaler)
print("Accuracy DecisionTreeClassifier with optimal hyperparameters:", accuracy_score(y_test, DecisionTreeGSCV_Predict) * 100)

DecisionTreeClassifier(max_depth=4, min_samples_leaf=0.030000000000000006,
                       min_samples_split=0.09000000000000001)
Accuracy DecisionTreeClassifier with optimal hyperparameters: 72.86245353159852


**RandomForestClassifier**

In [18]:
RandomForestGSCV.fit(X_train_scaler, y_train)
print(RandomForestGSCV.best_estimator_)
RandomForestGSCV_Predict = RandomForestGSCV.predict(X_test_scaler)
print("Accuracy RandomForestClassifier with optimal hyperparameters:", accuracy_score(y_test, RandomForestGSCV_Predict) * 100)

RandomForestClassifier(n_estimators=26)
Accuracy RandomForestClassifier with optimal hyperparameters: 75.8364312267658
